In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDRegressor as sgd

sns.set_style("whitegrid")
plt.rc('font', family='Times New Roman', size=18)

In [ ]:

df = pd.read_csv('sf_meta_dt.csv', parse_dates=['date'], index_col=['date', 'time'])
df = df.reset_index(drop=False)
date = df.date.tolist()
time = df.time.tolist()
df = df.drop(['date', 'time'], axis=1)
# df = df[df.Power > 0]
df = df.drop([

    'pvsim',
    'Lag_24',
    'Lag_25',
    'Lag_48',
    'Lag_72',

    'solar_azimuth',
    'solar_zenith',
    'solar_elevation',
    'solar_time',

    'shortwave_radiation',
    'direct_radiation',
    'diffuse_radiation',
    'direct_normal_irradiance',

    'weathercode',

    'temperature_2m',
    'dewpoint_2m',

    'relativehumidity_2m',
    'surface_pressure',

    'windspeed_10m',
    'winddirection_10m',

    'cloudcover',
    'cloudcover_low',
    'cloudcover_mid',
    'cloudcover_high',
    'cloud_radiation',

    'precipitation',
    'rain',
    'snowfall',
    'pred_lstm',
], axis=1)
print(df.columns)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = scaler.fit_transform(df.values)

y = scaled_df[:, 0]
y_max = np.max(y)
x = scaled_df[:, 1:]

train_size = int(len(x) * 0.80)

X_train, X_test, date_test, time_test = x[0:train_size], x[train_size:], date[train_size:], time[train_size:]
Y_train, Y_test = y[0:train_size], y[train_size:]
print(np.shape(df))
print(np.shape(X_test))

In [ ]:
params_sgd = {
    'loss': 'huber',
    'penalty': 'l1',
    'learning_rate': 'invscaling',
    'random_state': 42,
    'shuffle': False,
    # 'l1_ratio': 1,
    'alpha': 0.0016,
    'eta0': 0.3,
    'max_iter': 200,
    'power_t': 0.48,
    'tol': 0.0006,
}
model = sgd(**params_sgd)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

mse_scores = []
mae_scores = []
r2score = []

for train_index, test_index in kf.split(X_train):
    x_train, x_val = X_train[train_index], X_train[test_index]
    y_train, y_val = Y_train[train_index], Y_train[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, y_pred, squared=False)
    mae = mean_absolute_error(y_val, y_pred)
    r2sc = r2_score(y_val, y_pred)
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2score.append(r2sc)

In [ ]:

def mean_absolute_percentage(y_true, y_pred):
    # y_true, y_pred = list(map(float, y_true)), list(map(float, y_pred))
    # assert len(y_true) == len(y_pred), "Input lists must have the same length"
    ymean = np.mean(y_true)
    ape = [abs(y_true[i] - y_pred[i]) for i in range(len(y_true))]
    # Return the mean of the absolute percentage errors
    mape = (sum(ape) / len(y_true)) * 100
    mape = mape / ymean
    return mape


pred_train = model.predict(X_train)
print('train:')
print('mse score:', np.mean(mse_scores) / np.max(Y_train))
print('mae score:', np.mean(mae_scores) / np.max(Y_train))
print('r2 score:', np.mean(r2score))

# print('traini:')
# print('nRmse score:', mean_squared_error(Y_train, pred_train, squared=False)/np.max(Y_train))
# print('nmae score:', mean_absolute_error(Y_train, pred_train)/np.max(Y_train))
# print('r2 score:', r2_score(Y_train, pred_train))
# print('mape:', mean_absolute_percentage(Y_train, pred_train))

pred_test = model.predict(X_test)
# pred_test, ystd = model.predict(X_test)  #, return_std=True
x1 = 100
x2 = 400

print('test:')
print('nRmse score:', mean_squared_error(Y_test, pred_test, squared=False) / np.max(Y_test))
print('nmae score:', mean_absolute_error(Y_test, pred_test) / np.max(Y_test))
print('r2 score:', r2_score(Y_test, pred_test))
print('mape:', mean_absolute_percentage(Y_test, pred_test))

# plt.figure(figsize=(10, 6))
# plt.plot(Y_test[x1:x2])
# plt.plot(pred_test[x1:x2])
# plt.fill_between(range(0, x2-x1), pred_test[x1:x2] - ystd[x1:x2], pred_test[x1:x2] + ystd[x1:x2], color="pink", alpha=0.5, label="predict std")
# plt.show()

avg = (X_test[:, -2] + X_test[:, -1]) / 2

print('avg:')
print('nmse score:', mean_squared_error(Y_test, avg, squared=False) / np.max(Y_test))
print('nmae score:', mean_absolute_error(Y_test, avg) / np.max(Y_test))
print('r2 score:', r2_score(Y_test, avg))
print('mape:', mean_absolute_percentage(Y_test, avg))

print('lgbm:')
print('nmse score:', mean_squared_error(Y_test, X_test[:, -2], squared=False) / np.max(Y_test))
print('nmae score:', mean_absolute_error(Y_test, X_test[:, -2]) / np.max(Y_test))
print('r2 score:', r2_score(Y_test, X_test[:, -2]))
print('mape:', mean_absolute_percentage(Y_test, X_test[:, -2]))

print('saed:')
print('nmse score:', mean_squared_error(Y_test, X_test[:, -1], squared=False) / np.max(Y_test))
print('nmae score:', mean_absolute_error(Y_test, X_test[:, -1]) / np.max(Y_test))
print('r2 score:', r2_score(Y_test, X_test[:, -1]))
print('mape:', mean_absolute_percentage(Y_test, X_test[:, -1]))

y_persist = [0] * len(Y_test)

for i in range(len(Y_test) - 24):
    y_persist[i + 24] = Y_test[i]

# print('persistance:')
# print('nmse score:', mean_squared_error(Y_test, y_persist, squared=False) / np.max(Y_test))
# print('nmae score:', mean_absolute_error(Y_test, y_persist) / np.max(Y_test))
# print('r2 score:', r2_score(Y_test, y_persist))
# print('mape:', mean_absolute_percentage(Y_test, y_persist))

res_test = np.concatenate((Y_test.reshape(-1, 1), X_test), axis=1)
res_pred_test = np.concatenate((pred_test.reshape(-1, 1), X_test), axis=1)
res_test = scaler.inverse_transform(res_test)
res_pred_test = scaler.inverse_transform(res_pred_test)

Y_test = res_test[:, 0]
pred_ts = res_pred_test[:, 0]
pred_ts[pred_ts < 0] = 0
pred_ts_ex = np.concatenate(([0, 0, 0, 0], pred_ts), axis=0)
Y_test_ex = np.concatenate(([0, 0, 0, 0], Y_test), axis=0)
# pred_ts[Y_test < 50] = 0

In [ ]:
np.shape(Y_test_ex)

In [ ]:

## FIG 11
plt.figure(figsize=(22, 6))
plt.plot(Y_test, color='black')
plt.plot(pred_ts, color='red')
plt.ylabel('Solar power (W)')
plt.legend(['Actual', 'Forecast'])
plt.xlabel('Hour')
plt.xlim(-1, 2570)
# plt.savefig(f'figures/all_days.svg')
# plt.show()


## Fig 12
# for i, j in zip(range(0, len(Y_test_ex) - 24, 24), range(24, len(Y_test_ex), 24)):
#     plt.figure(figsize=(10, 6))
#     plt.plot(Y_test_ex[i:j], color='black')
#     plt.plot(pred_ts_ex[i:j], '--', color='red')
#     sns.despine()
#     plt.xlabel('Hour')
#     plt.ylabel('Solar power (W)')
#     plt.xlim(0, 23)
#     plt.legend(['Actual', 'Forecast'])
#     # plt.savefig(f'figures/hour_{i}.png')
#     plt.show()
#     # plt.close()

## Fig 13
y_test_sum = []
y_pred_sum = []
for i, j in zip(range(0, len(Y_test_ex) - 24, 24), range(24, len(Y_test_ex), 24)):
    y_test_sum.append((np.sum(Y_test_ex[i:j] / 1000)))
    y_pred_sum.append((np.sum(pred_ts_ex[i:j] / 1000)))
# 
# print(np.shape(y_test_sum))

In [ ]:

# bar_width = 0.4
# bar_positions_1 = [i for i in range(len(y_test_sum))]
# bar_positions_2 = [i + bar_width for i in bar_positions_1]

plt.figure(figsize=(22, 6))
# plt.bar(bar_positions_1, y_test_sum, width=bar_width, color='black', alpha=0.9)
# plt.bar(bar_positions_2, y_pred_sum, width=bar_width, color='darkred', alpha=0.8)
plt.plot(y_test_sum, color='black', marker='o')
plt.plot(y_pred_sum, color='maroon', marker='s')
sns.despine()
plt.xlabel('Day')
plt.ylabel('Total daily harvested solar energy (kWh)')
plt.legend(['Actual', 'Forecast'])
plt.xlim(-0.5, 107.1)
# plt.savefig(f'figures/sum_all_days3.svg')
plt.show()



# filename = "sgdr_sf_0.pkl"
# joblib.dump(model, filename)
# print(f"Saved sgdr model as {filename}")


In [ ]:
summed_residuals  = []
for i in range(0, len(y_test_sum)):
    summed_residuals.append(np.abs(y_test_sum[i]-y_pred_sum[i]))

num_vars = len(y_test_sum)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles.append(angles[0])
y_test_sum.append(y_test_sum[0])
y_pred_sum.append(y_pred_sum[0])
summed_residuals.append(summed_residuals[0])

fig, ax = plt.subplots(figsize=(10,10), subplot_kw=dict(polar=True))
# ax.plot(angles,summed_residuals,linewidth=2,color='#E64E04')
ax.plot(angles, y_test_sum,linewidth=2, label='Power', color='#143841')  # Plot summed 'Power'
# ax.fill(angles, summed_power, alpha=0.25, color='#143841')   # Fill
ax.plot(angles, y_pred_sum,'--' , linewidth=2, label='Predictions',color='#E64E04')  # Plot summed 'pvsim'
# ax.fill(angles, summed_pvsim, alpha=0.25,color='#E64E04')   # Fill
ax.fill(angles, summed_residuals,color='#0C6A6F', label='Residuals')   # Fill

y_ticks = ax.get_yticks()
y_labels = ['' if i%2 else str(y_tick) for i, y_tick in enumerate(y_ticks[1:-1])]
ax.set_yticks(y_ticks[1:-1])
ax.set_yticklabels(y_labels)
ax.legend(loc='upper center',bbox_to_anchor=(0.5, 0.95))
ax.set_xticks(angles[::5])
ax.set_xticklabels([f"day_{i}" for i in range(0, num_vars, 5)]) 
plt.savefig('prediction_radar_plot.png', dpi=500)
plt.show()


In [ ]:
# Fig 12
for i, j in zip(range(0, len(Y_test_ex) - 24, 24), range(24, len(Y_test_ex), 24)):
    plt.figure(figsize=(10, 6))
    plt.plot(Y_test_ex[i:j], color='black')
    plt.plot(pred_ts_ex[i:j], '--', color='red')
    sns.despine()
    plt.xlabel('Hour')
    plt.ylabel('Solar power (W)')
    plt.xlim(0, 23)
    plt.legend(['Actual', 'Forecast'])
    # plt.savefig(f'figures/hour_{i}.png')
    plt.show()
    # plt.close()

In [37]:
pred_0 = pd.DataFrame()
pred_0['date'] = date_test
pred_0['time'] = time_test
pred_0['farm_pred'] = pred_ts
pred_0['farm_real'] = Y_test
pred_0['saed_pred'] = X_test[:,-2]
pred_0['lightgbm_pred'] = X_test[:,-1]
pred_0.to_csv('sf_pred_meta2.csv')

In [36]:
for i in range(0, len(Y_test_ex) - 120, 120):
    j = i + 120  # End index for the 5 day period
    plt.figure(figsize=(10, 6))
    plt.plot(Y_test_ex[i:j], color='#143841')  # Actual data for 5 days
    plt.plot(pred_ts_ex[i:j], '--', color='#E64E04')  # Forecast data for 5 days
    
    sns.despine()
    plt.xlabel('Day')
    plt.ylabel('Solar power (W)')
    plt.xlim(0, 119)  # Set the x-axis limit to show 5 days
    plt.xticks(ticks=range(0, 120, 24), labels=[1,2,3,4,5])  # Set x-ticks to show day intervals
    # day_labels = [f'Day {1+n}' for n in range(5)]
    # for n, label in enumerate(day_labels):
    #     plt.text(n, -0.5, label, ha='center', va='top', transform=plt.gca().transAxes)
    plt.legend(['Actual', 'Prediction'])
    # Uncomment the line below to save figures
    plt.savefig(f'result/hour_{i//24}_to_{j//24}.png', dpi=500)
    plt.close()
    # plt.show()

In [ ]:
# for i, (start, end) in enumerate(zip(df.index[0::120], df.index[120::120])):
#     fig, ax = plt.subplots(figsize=(12, 6))
#     ax.plot(df.loc[start:end, 'Power'], color='#143841')
#     # print(dataset.loc[start:end, 'Power'].values)
#     ax.plot(df.loc[start:end, 'pvsim'], '--', color='#E64E04')
#     # ax.set_title(f'PV power output, date: {start.strftime("%Y-%m-%d")} to {end.strftime("%Y-%m-%d")}',fontsize= 20)
#     # ax.set_xlabel('Time',fontsize= 20)
#     ax.set_ylabel('Power (w)', fontsize=20)
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
#     ax.xaxis.set_major_locator(mdates.DayLocator())
#     ax.set_xticks(ax.get_xticks() + 0.5)
#     plt.legend(['Actual power', 'Simulated power'], fontsize= 20)
#     ax.set_xlim(start, end)
#     # ax.grid(False)
#     sns. despine()
#     filename = f'PVlib_plot/nZEB_PV_{start.strftime("%Y%m%d")}_{end.strftime("%Y%m%d")}.png'
#     plt.savefig(filename, format='png', dpi=500)
#     # plt.show()
#     # if i==5:
#     #     break
#     plt.close()